In [74]:
#import

import bs4
import json
import time
import requests
import scipy as sc
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By


starting to build our dataset using selenium

the contest started in the year 1956 BUT only at 1957 they started to rank with points so this is where we'll start

In [75]:
def move_page_euroWorld(Syear):
    
    if Syear==2020:
        Syear=2021
        driver.find_element_by_xpath('//*[@id="page"]/div[2]/main/div/div[1]/div[2]/a[2]').click()

    driver.find_element_by_xpath('//*[@id="page"]/div[2]/main/div/div[1]/div[2]/a[2]').click()

In [76]:
#lists to crawl from eurovisionworld generalBoard
Syear=1957
place=[]
country=[]
song=[]
artist=[]
year=[]
point=[]
running=[]

one=[]
two=[]
three=[]
four=[]
five=[]
six=[]
seven=[]
eight=[]
ten=[]
twelve=[]
points_year=[]

#from wikipedia

driver=webdriver.Chrome()

driver.get("https://eurovisionworld.com/eurovision/1957")



in this part we will crawl from eurovisionworld in order to recive the following: place, country,song,artist,year,point,running


In [ ]:
#testing
scoreboard_Table =driver.find_element_by_xpath('//*[@id="scoreboard"]/div/div[1]/table')
for row in scoreboard_Table.find_elements_by_css_selector('tr'):
    asdasd


In [ ]:

while Syear<=2021:
    #taking data from the general board
    general_Table =driver.find_element_by_xpath('//*[@id="voting_table"]/table/tbody')
    #from the table 
    for row in general_Table.find_elements_by_css_selector('tr'):
        place.append(row.find_elements_by_tag_name('td')[0].text)
        country.append(row.find_elements_by_tag_name('td')[1].text)
        #spliting the song and artist from the same cell:
        song.append(row.find_elements_by_tag_name('td')[2].text.split('\n')[0])
        artist.append(row.find_elements_by_tag_name('td')[2].text.split('\n')[1])
        point.append(row.find_elements_by_tag_name('td')[3].text)
        running.append(row.find_elements_by_tag_name('td')[4].text)
        year.append(Syear)

    #taking data from the Scoreboard table
    



    Syear+=1
    move_page_euroWorld(Syear)
    if Syear==2020:
        Syear+=1

In [84]:
dict={'Country':country,'Song':song,'Artist':artist,'Year':year,'Points':point,'Running_Order':running,'Place':place }
df=pd.DataFrame(dict)
df["Points"] = pd.to_numeric(df["Points"], downcast="float")
driver.close
# df.to_csv('eurovision_dataset.csv')
df


,Country,Song,Artist,Year,Points,Running_Order,Place
0,Netherlands,Net Als Toen,Corry Brokken,1957,31.0,6,1
1,France,La Belle Amour,Paule Desjardins,1957,17.0,8,2
2,Denmark,Skibet Skal Sejle I Nat,Birthe Wilke & Gustav Winckler,1957,10.0,9,3
3,Luxembourg,Amours Mortes (Tant De Peine),Danièle Dupré,1957,8.0,2,4
4,Germany,"Telefon, Telefon",Margot Hielscher,1957,8.0,7,4
5,Italy,Corde Della Mia Chitarra,Nunzio Gallo,1957,7.0,4,6
6,United Kingdom,All,Patricia Bredin,1957,6.0,3,7
7,Belgium,Straatdeuntje,Bobbejaan Schoepen,1957,5.0,1,8
8,Switzerland,L'enfant Que J'étais,Lys Assia,1957,5.0,10,8
9,Austria,"Wohin, Kleines Pony",Bob Martin,1957,3.0,5,10
